In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
import zipfile
import os


zip_file_name = '/content/drive/MyDrive/ANSYS/VRL_challenge_PAR.zip'


extract_to = '/content/'

with zipfile.ZipFile(zip_file_name, 'r') as zip_ref:
    zip_ref.extractall(extract_to)
print("Files extracted to:", extract_to)
os.listdir(extract_to)

Files extracted to: /content/


['.config', 'drive', 'VRL_challenge_PAR', 'sample_data']

In [ ]:
import os
import torch
from PIL import Image
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
import torch.nn as nn
import torchvision.models as models
from torch.optim import Adam
from torch.optim.lr_scheduler import StepLR
from sklearn.metrics import accuracy_score, f1_score
import pandas as pd


class PAR_Dataset(Dataset):
    def __init__(self, image_dir, annotation_file, label_file, transform=None):
        self.image_dir = image_dir
        self.transform = transform
        self.annotations = self.load_annotations(annotation_file)
        self.labels = self.load_labels(label_file)

    def load_annotations(self, annotation_file):
        with open(annotation_file, 'r') as f:
            lines = f.readlines()
        annotations = []
        for line in lines:
            parts = line.strip().split()
            image_name = parts[0] + '.jpg'
            attributes = list(map(int, parts[1:]))
            annotations.append((image_name, attributes))
        return annotations

    def load_labels(self, label_file):
        with open(label_file, 'r') as f:
            labels = [line.strip() for line in f.readlines()]
        return labels

    def __len__(self):
        return len(self.annotations)

    def __getitem__(self, idx):
        image_name, attributes = self.annotations[idx]
        img_path = os.path.join(self.image_dir, image_name)
        image = Image.open(img_path).convert('RGB')
        if self.transform:
            image = self.transform(image)
        attributes = torch.tensor(attributes, dtype=torch.float32)
        return image, attributes


train_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

val_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

class PAR_Model(nn.Module):
    def __init__(self, num_attributes):
        super(PAR_Model, self).__init__()
        self.resnet = models.resnet50(weights=models.ResNet50_Weights.DEFAULT)
        self.resnet.fc = nn.Linear(self.resnet.fc.in_features, num_attributes)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        x = self.resnet(x)
        x = self.sigmoid(x)
        return x


num_epochs = 200
learning_rate = 0.0001
batch_size = 16


train_dataset = PAR_Dataset('/content/drive/MyDrive/ANSYS/VRL_challenge_PAR_new/images', '/content/drive/MyDrive/ANSYS/VRL_challenge_PAR_new/train.txt', '/content/drive/MyDrive/ANSYS/VRL_challenge_PAR_new/label.txt', transform=train_transform)
val_dataset = PAR_Dataset('/content/drive/MyDrive/ANSYS/VRL_challenge_PAR_new/validation_image', '/content/drive/MyDrive/ANSYS/VRL_challenge_PAR_new/validation.txt', '/content/drive/MyDrive/ANSYS/VRL_challenge_PAR_new/label.txt', transform=val_transform)

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

# Model, loss function, and optimizer
num_attributes = len(train_dataset.labels)
model = PAR_Model(num_attributes)
criterion = nn.BCELoss()
optimizer = Adam(model.parameters(), lr=learning_rate)
scheduler = StepLR(optimizer, step_size=7, gamma=0.1)  # Learning rate scheduler

# Training and validation loop
best_val_loss = float('inf')

for epoch in range(num_epochs):
    model.train()
    train_loss = 0.0
    for images, labels in train_loader:
        outputs = model(images)
        loss = criterion(outputs, labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        train_loss += loss.item()

    train_loss /= len(train_loader)

    model.eval()
    val_loss = 0.0
    all_labels = []
    all_outputs = []

    with torch.no_grad():
        for images, labels in val_loader:
            outputs = model(images)
            loss = criterion(outputs, labels)
            val_loss += loss.item()

            all_labels.extend(labels.cpu().numpy())
            all_outputs.extend(outputs.cpu().numpy())

    val_loss /= len(val_loader)
    scheduler.step()

    all_labels = torch.tensor(all_labels)
    all_outputs = torch.tensor(all_outputs)
    val_acc = accuracy_score(all_labels, all_outputs > 0.5)
    val_f1 = f1_score(all_labels, all_outputs > 0.5, average='weighted')

    print(f'Epoch [{epoch+1}/{num_epochs}], Train Loss: {train_loss:.4f}, Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.4f}, Val F1: {val_f1:.4f}')

    if val_loss < best_val_loss:
        best_val_loss = val_loss
        torch.save(model.state_dict(), 'par_model_resnet50_best.pth')

# Predict on test images and save results to CSV
def predict_on_images(model, image_folder, output_csv, transform):
    model.eval()
    image_paths = [os.path.join(image_folder, img) for img in os.listdir(image_folder) if img.endswith(('.jpg', '.jpeg', '.png'))]
    image_paths.sort(key=lambda x: int(os.path.splitext(os.path.basename(x))[0]))  # Sort by image name assuming names are like 1.jpg, 2.jpg, etc.

    predictions = []

    for image_path in image_paths:
        image = Image.open(image_path).convert('RGB')
        image = transform(image).unsqueeze(0)  # Add batch dimension

        with torch.no_grad():
            outputs = model(image)
            outputs = (outputs > 0.5).int().squeeze().tolist()  # Apply threshold

        image_name = os.path.basename(image_path)
        predictions.append([image_name] + outputs)

    # Save to CSV
    header = ['image_name'] + [f'attribute_{i}' for i in range(1, num_attributes + 1)]
    df = pd.DataFrame(predictions, columns=header)
    df.to_csv(output_csv, index=False)
    print(f"Predictions saved to {output_csv}")

# Path to the folder containing the test images
test_image_folder = '/content/drive/MyDrive/ANSYS/SCSPAR24_Testdata'  # Replace with your actual test image folder path
output_csv = '/content/predictions.csv'

# Load the best model for predictions
model.load_state_dict(torch.load('par_model_resnet50_best.pth'))

# Run predictions and save to CSV
predict_on_images(model, test_image_folder, output_csv, val_transform)


Downloading: "https://download.pytorch.org/models/resnet50-11ad3fa6.pth" to /root/.cache/torch/hub/checkpoints/resnet50-11ad3fa6.pth
100%|██████████| 97.8M/97.8M [00:01<00:00, 86.2MB/s]
<ipython-input-4-7fe9f971f46d>:130: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:274.)
  all_labels = torch.tensor(all_labels)


Epoch [1/200], Train Loss: 0.5511, Val Loss: 0.4683, Val Acc: 0.0000, Val F1: 0.1448
Epoch [2/200], Train Loss: 0.3641, Val Loss: 0.4439, Val Acc: 0.0000, Val F1: 0.1745
Epoch [3/200], Train Loss: 0.3109, Val Loss: 0.4279, Val Acc: 0.0000, Val F1: 0.2252
Epoch [4/200], Train Loss: 0.2814, Val Loss: 0.4115, Val Acc: 0.0000, Val F1: 0.2483
Epoch [5/200], Train Loss: 0.2555, Val Loss: 0.3985, Val Acc: 0.0000, Val F1: 0.2782
Epoch [6/200], Train Loss: 0.2328, Val Loss: 0.3964, Val Acc: 0.0000, Val F1: 0.3029
Epoch [7/200], Train Loss: 0.2116, Val Loss: 0.4047, Val Acc: 0.0000, Val F1: 0.3169
Epoch [8/200], Train Loss: 0.1949, Val Loss: 0.3942, Val Acc: 0.0000, Val F1: 0.3239
Epoch [9/200], Train Loss: 0.1901, Val Loss: 0.3875, Val Acc: 0.0000, Val F1: 0.3350
Epoch [10/200], Train Loss: 0.1884, Val Loss: 0.3872, Val Acc: 0.0000, Val F1: 0.3352
Epoch [11/200], Train Loss: 0.1893, Val Loss: 0.3891, Val Acc: 0.0000, Val F1: 0.3311
Epoch [12/200], Train Loss: 0.1848, Val Loss: 0.3871, Val Acc: 

In [ ]:
import os
import torch
from PIL import Image
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
import torch.nn as nn
import torchvision.models as models
from torch.optim import Adam
from torch.optim.lr_scheduler import StepLR
from sklearn.metrics import accuracy_score, f1_score
import pandas as pd


class PAR_Dataset(Dataset):
    def __init__(self, image_dir, annotation_file, label_file, transform=None):
        self.image_dir = image_dir
        self.transform = transform
        self.annotations = self.load_annotations(annotation_file)
        self.labels = self.load_labels(label_file)
        self.image_paths = {os.path.splitext(f)[0]: os.path.join(image_dir, f) for f in os.listdir(image_dir)}

    def load_annotations(self, annotation_file):
        with open(annotation_file, 'r') as f:
            lines = f.readlines()
        annotations = []
        for line in lines:
            parts = line.strip().split()
            image_name = parts[0]
            attributes = list(map(int, parts[1:]))
            annotations.append((image_name, attributes))
        return annotations

    def load_labels(self, label_file):
        with open(label_file, 'r') as f:
            labels = [line.strip() for line in f.readlines()]
        return labels

    def __len__(self):
        return len(self.annotations)

    def __getitem__(self, idx):
        image_name, attributes = self.annotations[idx]
        img_path = self.image_paths[image_name]
        image = Image.open(img_path).convert('RGB')
        if self.transform:
            image = self.transform(image)
        attributes = torch.tensor(attributes, dtype=torch.float32)
        return image, attributes


In [ ]:
train_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

val_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])


In [ ]:
class PAR_Model(nn.Module):
    def __init__(self, num_attributes):
        super(PAR_Model, self).__init__()
        self.resnet = models.resnet50(weights=models.ResNet50_Weights.DEFAULT)
        self.resnet.fc = nn.Linear(self.resnet.fc.in_features, num_attributes)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        x = self.resnet(x)
        x = self.sigmoid(x)
        return x


In [ ]:
num_epochs = 2
learning_rate = 0.0001
batch_size = 16

train_dataset = PAR_Dataset('/content/drive/MyDrive/ANSYS/VRL_challenge_PAR_new/images', '/content/drive/MyDrive/ANSYS/VRL_challenge_PAR_new/train.txt', '/content/drive/MyDrive/ANSYS/VRL_challenge_PAR_new/label.txt', transform=train_transform)
val_dataset = PAR_Dataset('/content/drive/MyDrive/ANSYS/VRL_challenge_PAR_new/validation_image', '/content/drive/MyDrive/ANSYS/VRL_challenge_PAR_new/validation.txt', '/content/drive/MyDrive/ANSYS/VRL_challenge_PAR_new/label.txt', transform=val_transform)

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

num_attributes = len(train_dataset.labels)
model = PAR_Model(num_attributes)
criterion = nn.BCELoss()
optimizer = Adam(model.parameters(), lr=learning_rate)
scheduler = StepLR(optimizer, step_size=7, gamma=0.1)

best_val_loss = float('inf')

for epoch in range(num_epochs):
    model.train()
    train_loss = 0.0
    for images, labels in train_loader:
        outputs = model(images)
        loss = criterion(outputs, labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        train_loss += loss.item()

    train_loss /= len(train_loader)

    model.eval()
    val_loss = 0.0
    all_labels = []
    all_outputs = []

    with torch.no_grad():
        for images, labels in val_loader:
            outputs = model(images)
            loss = criterion(outputs, labels)
            val_loss += loss.item()

            all_labels.extend(labels.cpu().numpy())
            all_outputs.extend(outputs.cpu().numpy())

    val_loss /= len(val_loader)
    scheduler.step()

    all_labels = torch.tensor(all_labels)
    all_outputs = torch.tensor(all_outputs)
    val_acc = accuracy_score(all_labels, all_outputs > 0.5)
    val_f1 = f1_score(all_labels, all_outputs > 0.5, average='weighted')

    print(f'Epoch [{epoch+1}/{num_epochs}], Train Loss: {train_loss:.4f}, Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.4f}, Val F1: {val_f1:.4f}')

    if val_loss < best_val_loss:
        best_val_loss = val_loss
        torch.save(model.state_dict(), 'par_model_resnet50_best.pth')


Downloading: "https://download.pytorch.org/models/resnet50-11ad3fa6.pth" to /root/.cache/torch/hub/checkpoints/resnet50-11ad3fa6.pth
100%|██████████| 97.8M/97.8M [00:01<00:00, 63.0MB/s]
<ipython-input-6-9f1cb33dd7a9>:51: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:274.)
  all_labels = torch.tensor(all_labels)


Epoch [1/2], Train Loss: 0.5685, Val Loss: 0.4937, Val Acc: 0.0000, Val F1: 0.1609
Epoch [2/2], Train Loss: 0.3727, Val Loss: 0.4588, Val Acc: 0.0000, Val F1: 0.1761


In [ ]:
class PAR_Dataset(Dataset):
    def __init__(self, image_dir, annotation_file, label_file, transform=None):
        self.image_dir = image_dir
        self.transform = transform
        self.annotations = self.load_annotations(annotation_file)
        self.labels = self.load_labels(label_file)
        self.image_paths = {os.path.splitext(f)[0]: os.path.join(image_dir, f) for f in os.listdir(image_dir)}

    def load_annotations(self, annotation_file):
        with open(annotation_file, 'r') as f:
            lines = f.readlines()
        annotations = []
        for line in lines:
            parts = line.strip().split()
            image_name = parts[0]
            attributes = list(map(int, parts[1:]))
            annotations.append((image_name, attributes))
        return annotations

    def load_labels(self, label_file):
        with open(label_file, 'r') as f:
            labels = [line.strip() for line in f.readlines()]
        return labels

    def __len__(self):
        return len(self.annotations)

    def __getitem__(self, idx):
        image_name, attributes = self.annotations[idx]
        img_path = self.image_paths[image_name]
        image = Image.open(img_path).convert('RGB')
        if self.transform:
            image = self.transform(image)
        attributes = torch.tensor(attributes, dtype=torch.float32)
        return image, attributes


In [ ]:
import numpy as np

num_epochs = 1
learning_rate = 0.0001
batch_size = 16

train_dataset = PAR_Dataset('/content/drive/MyDrive/ANSYS/VRL_challenge_PAR_new/images', '/content/drive/MyDrive/ANSYS/VRL_challenge_PAR_new/train.txt', '/content/drive/MyDrive/ANSYS/VRL_challenge_PAR_new/label.txt', transform=train_transform)
val_dataset = PAR_Dataset('/content/drive/MyDrive/ANSYS/VRL_challenge_PAR_new/validation_image', '/content/drive/MyDrive/ANSYS/VRL_challenge_PAR_new/validation.txt', '/content/drive/MyDrive/ANSYS/VRL_challenge_PAR_new/label.txt', transform=val_transform)

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

num_attributes = len(train_dataset.labels)
model = PAR_Model(num_attributes)
criterion = nn.BCELoss()
optimizer = Adam(model.parameters(), lr=learning_rate)
scheduler = StepLR(optimizer, step_size=7, gamma=0.1)

best_val_loss = float('inf')

for epoch in range(num_epochs):
    model.train()
    train_loss = 0.0
    for images, labels in train_loader:
        outputs = model(images)
        loss = criterion(outputs, labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        train_loss += loss.item()

    train_loss /= len(train_loader)

    model.eval()
    val_loss = 0.0
    all_labels = []
    all_outputs = []

    with torch.no_grad():
        for images, labels in val_loader:
            outputs = model(images)
            loss = criterion(outputs, labels)
            val_loss += loss.item()

            all_labels.extend(labels.cpu().numpy())
            all_outputs.extend(outputs.cpu().numpy())

    val_loss /= len(val_loader)
    scheduler.step()

    all_labels = np.array(all_labels)
    all_outputs = np.array(all_outputs)
    val_acc = accuracy_score(all_labels, all_outputs > 0.1)
    val_f1 = f1_score(all_labels, all_outputs > 0.1, average='weighted')

    print(f'Epoch [{epoch+1}/{num_epochs}], Train Loss: {train_loss:.4f}, Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.4f}, Val F1: {val_f1:.4f}')

    if val_loss < best_val_loss:
        best_val_loss = val_loss
        torch.save(model.state_dict(), 'par_model_resnet50_best.pth')


Epoch [1/1], Train Loss: 0.5550, Val Loss: 0.4800, Val Acc: 0.0000, Val F1: 0.1768


In [ ]:
# Debugging: Print a sample of predictions and corresponding labels
sample_size = 5
print("Sample predictions and labels:")
for i in range(sample_size):
    print(f"Prediction: {all_outputs[i]}, Label: {all_labels[i]}")


Sample predictions and labels:
Prediction: [0.09589572 0.09972013 0.08869521 0.04577098 0.05574826 0.22703566
 0.0638983  0.18464483 0.08444769 0.46409252 0.14015554 0.05485197
 0.0455873  0.04473143 0.07262193 0.04236887 0.0755322  0.04632498
 0.0806694  0.05131893 0.0498988  0.06160627 0.28105184 0.08366938
 0.18467076 0.17501098 0.07786711 0.06601954 0.03573979 0.61859727
 0.12986138 0.03897052 0.08915626 0.8075033  0.15205511 0.09102902
 0.17420816 0.10386563 0.07306469 0.3025067  0.05996171 0.48080343
 0.23239644 0.4288802  0.05478685 0.5398211  0.60016936 0.33856764
 0.10845541], Label: [0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 1.
 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 1. 1. 0. 0. 1. 0. 0. 1. 1. 0.
 0.]
Prediction: [0.14021645 0.16847602 0.10646795 0.06631079 0.10488731 0.3600579
 0.09023795 0.24833031 0.1233279  0.60722923 0.22356336 0.07534391
 0.06988175 0.09166374 0.13204384 0.06131313 0.1483699  0.09485362
 0.10092746 0.0943982  0.10530324 0.0

In [ ]:
import os
import torch
from PIL import Image
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
import torch.nn as nn
import torchvision.models as models
from torch.optim import Adam
from torch.optim.lr_scheduler import StepLR
from sklearn.metrics import accuracy_score, f1_score
import pandas as pd
import numpy as np

class PAR_Dataset(Dataset):
    def __init__(self, image_dir, annotation_file, label_file, transform=None):
        self.image_dir = image_dir
        self.transform = transform
        self.annotations = self.load_annotations(annotation_file)
        self.labels = self.load_labels(label_file)
        self.image_paths = {os.path.splitext(f)[0]: os.path.join(image_dir, f) for f in os.listdir(image_dir)}

    def load_annotations(self, annotation_file):
        with open(annotation_file, 'r') as f:
            lines = f.readlines()
        annotations = []
        for line in lines,:
            parts = line.strip().split()
            image_name = parts[0]
            attributes = list(map(int, parts[1:]))
            annotations.append((image_name, attributes))
        return annotations

    def load_labels(self, label_file):
        with open(label_file, 'r') as f:
            labels = [line.strip() for line in f.readlines()]
        return labels

    def __len__(self):
        return len(self.annotations)

    def __getitem__(self, idx):
        image_name, attributes = self.annotations[idx]
        img_path = self.image_paths[image_name]
        image = Image.open(img_path).convert('RGB')
        if self.transform:
            image = self.transform(image)
        attributes = torch.tensor(attributes, dtype=torch.float32)
        return image, attributes, image_name

train_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

val_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

class PAR_Model(nn.Module):
    def __init__(self, num_attributes):
        super(PAR_Model, self).__init__()
        self.resnet = models.resnet50(weights=models.ResNet50_Weights.DEFAULT)
        self.resnet.fc = nn.Linear(self.resnet.fc.in_features, num_attributes)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        x = self.resnet(x)
        x = self.sigmoid(x)
        return x

num_epochs = 1
learning_rate = 0.0001
batch_size = 16

train_dataset = PAR_Dataset('/content/drive/MyDrive/ANSYS/VRL_challenge_PAR_new/images', '/content/drive/MyDrive/ANSYS/VRL_challenge_PAR_new/train.txt', '/content/drive/MyDrive/ANSYS/VRL_challenge_PAR_new/label.txt', transform=train_transform)
val_dataset = PAR_Dataset('/content/drive/MyDrive/ANSYS/VRL_challenge_PAR_new/validation_image', '/content/drive/MyDrive/ANSYS/VRL_challenge_PAR_new/validation.txt', '/content/drive/MyDrive/ANSYS/VRL_challenge_PAR_new/label.txt', transform=val_transform)

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

# Model, loss function, and optimizer
num_attributes = len(train_dataset.labels)
model = PAR_Model(num_attributes)
criterion = nn.BCELoss()
optimizer = Adam(model.parameters(), lr=learning_rate)
scheduler = StepLR(optimizer, step_size=7, gamma=0.1)

best_val_loss = float('inf')

for epoch in range(num_epochs):
    model.train()
    train_loss = 0.0
    for images, labels, _ in train_loader:  # Ignore image names during training
        outputs = model(images)
        loss = criterion(outputs, labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        train_loss += loss.item()

    train_loss /= len(train_loader)

    model.eval()
    val_loss = 0.0
    all_labels = []
    all_outputs = []
    image_names = []

    with torch.no_grad():
        for images, labels, img_names in val_loader:
            outputs = model(images)
            loss = criterion(outputs, labels)
            val_loss += loss.item()

            all_labels.extend(labels.cpu().numpy())
            all_outputs.extend(outputs.cpu().numpy())
            image_names.extend(img_names)

    val_loss /= len(val_loader)
    scheduler.step()

    all_labels = np.array(all_labels)
    all_outputs = np.array(all_outputs)
    val_acc = accuracy_score(all_labels, all_outputs > 0.5)
    val_f1 = f1_score(all_labels, all_outputs > 0.5, average='weighted')

    print(f'Epoch [{epoch+1}/{num_epochs}], Train Loss: {train_loss:.4f}, Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.4f}, Val F1: {val_f1:.4f}')

    # Debugging: Print a sample of predictions, corresponding labels, and image names
    sample_size = 5
    print("Sample predictions, labels, and image names:")
    for i in range(sample_size):
        print(f"Image: {image_names[i]}, Prediction: {all_outputs[i]}, Label: {all_labels[i]}")

    if val_loss < best_val_loss:
        best_val_loss = val_loss
        torch.save(model.state_dict(), 'par_model_resnet50_best.pth')


AttributeError: 'list' object has no attribute 'strip'

In [ ]:
import os
import torch
from PIL import Image
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
import torch.nn as nn
import torchvision.models as models
from torch.optim import Adam
from torch.optim.lr_scheduler import StepLR
from sklearn.metrics import accuracy_score, f1_score
import pandas as pd
import numpy as np

class PAR_Dataset(Dataset):
    def __init__(self, image_dir, annotation_file, label_file, transform=None):
        self.image_dir = image_dir
        self.transform = transform
        self.annotations = self.load_annotations(annotation_file)
        self.labels = self.load_labels(label_file)
        self.image_paths = {os.path.splitext(f)[0]: os.path.join(image_dir, f) for f in os.listdir(image_dir)}

    def load_annotations(self, annotation_file):
        with open(annotation_file, 'r') as f:
            lines = f.readlines()
        annotations = []
        for line in lines:
            parts = line.strip().split()
            image_name = parts[0]
            attributes = list(map(int, parts[1:]))
            annotations.append((image_name, attributes))
        return annotations

    def load_labels(self, label_file):
        with open(label_file, 'r') as f:
            labels = [line.strip() for line in f.readlines()]
        return labels

    def __len__(self):
        return len(self.annotations)

    def __getitem__(self, idx):
        image_name, attributes = self.annotations[idx]
        img_path = self.image_paths[image_name]
        image = Image.open(img_path).convert('RGB')
        if self.transform:
            image = self.transform(image)
        attributes = torch.tensor(attributes, dtype=torch.float32)
        return image, attributes, image_name

train_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

val_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

class PAR_Model(nn.Module):
    def __init__(self, num_attributes):
        super(PAR_Model, self).__init__()
        self.resnet = models.resnet50(weights=models.ResNet50_Weights.DEFAULT)
        self.resnet.fc = nn.Linear(self.resnet.fc.in_features, num_attributes)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        x = self.resnet(x)
        x = self.sigmoid(x)
        return x

num_epochs = 20
learning_rate = 0.0001
batch_size = 16

train_dataset = PAR_Dataset('/content/drive/MyDrive/ANSYS/VRL_challenge_PAR_new/images', '/content/drive/MyDrive/ANSYS/VRL_challenge_PAR_new/train.txt', '/content/drive/MyDrive/ANSYS/VRL_challenge_PAR_new/label.txt', transform=train_transform)
val_dataset = PAR_Dataset('/content/drive/MyDrive/ANSYS/VRL_challenge_PAR_new/validation_image', '/content/drive/MyDrive/ANSYS/VRL_challenge_PAR_new/validation.txt', '/content/drive/MyDrive/ANSYS/VRL_challenge_PAR_new/label.txt', transform=val_transform)

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

# Model, loss function, and optimizer
num_attributes = len(train_dataset.labels)
model = PAR_Model(num_attributes)
criterion = nn.BCELoss()
optimizer = Adam(model.parameters(), lr=learning_rate)
scheduler = StepLR(optimizer, step_size=7, gamma=0.1)

best_val_loss = float('inf')

for epoch in range(num_epochs):
    model.train()
    train_loss = 0.0
    for images, labels, _ in train_loader:  # Ignore image names during training
        outputs = model(images)
        loss = criterion(outputs, labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        train_loss += loss.item()

    train_loss /= len(train_loader)

    model.eval()
    val_loss = 0.0
    all_labels = []
    all_outputs = []
    image_names = []

    with torch.no_grad():
        for images, labels, img_names in val_loader:
            outputs = model(images)
            loss = criterion(outputs, labels)
            val_loss += loss.item()

            all_labels.extend(labels.cpu().numpy())
            all_outputs.extend(outputs.cpu().numpy())
            image_names.extend(img_names)

    val_loss /= len(val_loader)
    scheduler.step()

    all_labels = np.array(all_labels)
    all_outputs = np.array(all_outputs)
    val_acc = accuracy_score(all_labels, all_outputs > 0.5)
    val_f1 = f1_score(all_labels, all_outputs > 0.5, average='weighted')

    print(f'Epoch [{epoch+1}/{num_epochs}], Train Loss: {train_loss:.4f}, Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.4f}, Val F1: {val_f1:.4f}')

    # Debugging: Print a sample of predictions, corresponding labels, and image names
    sample_size = 5
    print("Sample predictions, labels, and image names:")
    for i in range(sample_size):
        print(f"Image: {image_names[i]}, Prediction: {all_outputs[i]}, Label: {all_labels[i]}")

    if val_loss < best_val_loss:
        best_val_loss = val_loss
        torch.save(model.state_dict(), 'par_model_resnet50_best.pth')


Epoch [1/20], Train Loss: 0.5614, Val Loss: 0.4663, Val Acc: 0.0000, Val F1: 0.1556
Sample predictions, labels, and image names:
Image: 415, Prediction: [0.13004978 0.11572918 0.11191966 0.14708978 0.07257079 0.1762462
 0.10915355 0.17968765 0.1335721  0.49178645 0.11886345 0.06997617
 0.08680376 0.10277146 0.07320983 0.12374671 0.12134406 0.09477093
 0.10212959 0.09244028 0.11991879 0.07119675 0.3126276  0.08419721
 0.28306064 0.11264478 0.11020786 0.11107735 0.12094741 0.7257078
 0.07665201 0.06653494 0.06685701 0.79241073 0.16382715 0.08299407
 0.2149164  0.07412721 0.07453158 0.33236122 0.10644775 0.4403955
 0.15318787 0.56889087 0.16224137 0.5491767  0.4973974  0.26749355
 0.12166075], Label: [0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 1.
 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 1. 1. 0. 0. 1. 0. 0. 1. 1. 0.
 0.]
Image: 416, Prediction: [0.21046665 0.14980501 0.19944511 0.21257415 0.22670232 0.27365467
 0.20008762 0.27162257 0.18007429 0.5471615  0.2349

KeyboardInterrupt: 